<a href="https://colab.research.google.com/github/sat0urn/CV_Assignments/blob/main/Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Loading Dataset**

In [10]:
import numpy as np
import os
import cv2
import random
import math
import pdb

# Loading Dataset

def load_dataset(data_dir):
    classes = os.listdir(data_dir)

    images = []

    labels = []

    for class_name in classes:
      class_dir = os.path.join(data_dir, class_name)
      for image_file in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_file)
        image = cv2.imread(image_path)
        image = cv2.resize(image, (128, 128))
        images.append(image)
        labels.append(class_name)

    return np.array(images), np.array(labels)

data_dir = '/content/drive/MyDrive/Colab Notebooks/Agricultural-crops'

images, labels = load_dataset(data_dir)

# **Softmax Regression Algorithm**

In [22]:
# Softmax Regression Algorithm
class SoftmaxRegression:
    def __init__(self, learning_rate=0.01, n_iterations=100, regularization='L2', lambda_reg=0.01):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.regularization = regularization
        self.lambda_reg = lambda_reg

    def fit(self, X, y):
        num_samples, num_features = X.shape
        num_classes = len(np.unique(y))
        self.theta = np.random.rand(num_features, num_classes)

        for i in range(self.n_iterations):
            scores = X.dot(self.theta)
            exp_scores = np.exp(scores)
            probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

            delta = probs
            delta[range(num_samples), y] -= 1
            delta /= num_samples

            dtheta = X.T.dot(delta)

            if self.regularization == 'L2':
                dtheta += self.lambda_reg * self.theta
            elif self.regularization == 'L1':
                dtheta += self.lambda_reg * np.sign(self.theta)

            self.theta -= self.learning_rate * dtheta

    def predict(self, X):
        scores = X.dot(self.theta)
        return np.argmax(scores, axis=1)

# **SVM Algorithm**

In [13]:
# SVM Algorithm
class SVM:
    def __init__(self, learning_rate=0.01, n_iterations=100, regularization='L2', lambda_reg=0.01):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.regularization = regularization
        self.lambda_reg = lambda_reg

    def fit(self, X, y):
        num_samples, num_features = X.shape
        num_classes = len(np.unique(y))
        self.theta = np.random.rand(num_features, num_classes)

        for i in range(self.n_iterations):
            scores = X.dot(self.theta)
            correct_scores = scores[range(num_samples), y]
            margins = np.maximum(0, scores - correct_scores[:, np.newaxis] + 1)
            margins[range(num_samples), y] = 0
            loss = np.sum(margins) / num_samples

            # Compute gradients
            binary = margins
            binary[margins > 0] = 1
            row_sum = np.sum(binary, axis=1)
            binary[range(num_samples), y] = -row_sum
            dtheta = X.T.dot(binary)

            if self.regularization == 'L2':
                dtheta += self.lambda_reg * self.theta
            elif self.regularization == 'L1':
                dtheta += self.lambda_reg * np.sign(self.theta)

            self.theta -= self.learning_rate * dtheta

    def predict(self, X):
        scores = X.dot(self.theta)
        return np.argmax(scores, axis=1)

# **Divide into different sets (train, validation, test)**

In [14]:
# Divide data into train, validation, and test sets
def custom_train_val_test_split(X, y, val_size=0.1, test_size=0.1):
    num_samples = X.shape[0]
    val_size = int(num_samples * val_size)
    test_size = int(num_samples * test_size)

    indices = np.random.permutation(num_samples)

    val_indices = indices[:val_size]
    test_indices = indices[val_size:val_size+test_size]
    train_indices = indices[val_size+test_size:]

    X_val, y_val = X[val_indices], y[val_indices]
    X_test, y_test = X[test_indices], y[test_indices]
    X_train, y_train = X[train_indices], y[train_indices]

    return X_train, y_train, X_val, y_val, X_test, y_test

# Split data into train, validation, and test sets
X_train, y_train, X_val, y_val, X_test, y_test = custom_train_val_test_split(images, labels, val_size=0.1, test_size=0.1)

# **Split train into train and validation sets for cross-validation**

In [15]:
# Further split train into train and validation sets for cross-validation
def k_fold_split(X, y, k=5):
    fold_size = len(X) // k
    for i in range(k):
        start = i * fold_size
        end = (i + 1) * fold_size
        X_val = X[start:end]
        y_val = y[start:end]
        X_train = np.concatenate([X[:start], X[end:]])
        y_train = np.concatenate([y[:start], y[end:]])
        yield X_train, y_train, X_val, y_val

# **Calculate Accuracy**

In [16]:
def calculate_accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

# **Initialization Models and Accuracy Evaluation**

In [ ]:
# Initialize models
softmax_model = SoftmaxRegression(learning_rate=0.01, n_iterations=100, regularization='L2', lambda_reg=0.01)
svm_model = SVM(learning_rate=0.01, n_iterations=1000, regularization='L2', lambda_reg=0.01)

best_accuracy = -1
best_model = None

for X_train_fold, y_train_fold, X_val_fold, y_val_fold in k_fold_split(X_train, y_train, k=5):
    # Train Softmax Regression
    softmax_model.fit(X_train_fold, y_train_fold)
    softmax_predictions = softmax_model.predict(X_val_fold)
    softmax_accuracy = calculate_accuracy(y_val_fold, softmax_predictions)

    # Train SVM
    svm_model.fit(X_train_fold, y_train_fold)
    svm_predictions = svm_model.predict(X_val_fold)
    svm_accuracy = calculate_accuracy(y_val_fold, svm_predictions)

    if softmax_accuracy > best_accuracy:
        best_accuracy = softmax_accuracy
        best_model = "Softmax Regression"

    if svm_accuracy > best_accuracy:
        best_accuracy = svm_accuracy
        best_model = "SVM"

# Evaluate the best model on the test set
if best_model == "Softmax Regression":
    softmax_model.fit(X_train, y_train)
    test_predictions = softmax_model.predict(X_test)
elif best_model == "SVM":
    svm_model.fit(X_train, y_train)
    test_predictions = svm_model.predict(X_test)

test_accuracy = calculate_accuracy(y_test, test_predictions)

print("Best Model:", best_model)
print("Test Accuracy:", test_accuracy)